# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - API Development

**Subtopic** - Building a RESTful API with Flask

**Use Case** - Develop a Python module that reads a CSV file and transforms it into a JSON object

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

## **Problem Overview**

You are tasked with building a robust and extensible Python module named `CSVTransformer` that reads a CSV file and transforms its content into a structured JSON object. This transformation must support:

1. **Schema Mapping**: Ability to convert field names using a provided mapping.
2. **Data Typing**: Enforce type conversion based on the mapped schema.
3. **Mode Handling**: Support different transformation modes like:

   * `flat`: Each CSV row becomes a standalone JSON object.
   * `grouped`: Rows are grouped by a specified field.
   * `nested`: Nested objects are created based on column prefixes.
4. **Error Handling**: Gracefully handle bad rows, missing columns, and type mismatches with options to skip, log, or raise.

This module should work as a standalone component that can be easily integrated into larger data pipelines.


## **Input Format**

* A **CSV file** with headers in the first row. The values may be strings, numbers, booleans, or ISO8601-formatted timestamps.
* A **schema mapping dictionary** (`Dict[str, Dict]`) specifying:

  * `target_field`: the new name of the column
  * `type`: the expected Python type (`str`, `int`, `float`, `bool`, `datetime`)
* A **mode string** (`str`) from: `"flat"`, `"grouped"`, `"nested"`
* Optional parameters:

  * `group_by`: required only in `grouped` mode
  * `delimiter`: defaults to `,`
  * `on_error`: one of `"raise"`, `"skip"`, or `"log"`


## **Output Format**

* A **JSON object** (`Dict` or `List[Dict]`) representing the transformed data.

  * For `flat` mode: a list of JSON objects
  * For `grouped` mode: a dictionary mapping `group_by` values to lists of objects
  * For `nested` mode: a list of JSON objects with nested structures based on prefixes in headers (e.g., `address.street` becomes nested under `address`)


## **Examples**

### Input CSV (`users.csv`):

```
id,name,age,address.street,address.city,active
1,Alice,30,123 Main St,Metropolis,true
2,Bob,25,456 Elm St,Gotham,false
```

### Schema Mapping:

```python
{
  "id": {"target_field": "userId", "type": int},
  "name": {"target_field": "fullName", "type": str},
  "age": {"target_field": "age", "type": int},
  "address.street": {"target_field": "address.street", "type": str},
  "address.city": {"target_field": "address.city", "type": str},
  "active": {"target_field": "isActive", "type": bool}
}
```

### Mode: `"nested"`

### Output:

```json
[
  {
    "userId": 1,
    "fullName": "Alice",
    "age": 30,
    "address": {
      "street": "123 Main St",
      "city": "Metropolis"
    },
    "isActive": true
  },
  {
    "userId": 2,
    "fullName": "Bob",
    "age": 25,
    "address": {
      "street": "456 Elm St",
      "city": "Gotham"
    },
    "isActive": false
  }
]
```


# Requirements


## **Explicit Requirements**

* The module must expose a public interface to:

  * Load CSV content from a file path
  * Accept and validate schema mapping
  * Choose transformation mode (`flat`, `grouped`, or `nested`)
  * Handle errors based on `on_error` setting
* Output should be a Python object (not written to file)
* Columns not in the schema must be ignored


## **Implicit Requirements**

* The module should be written in a way that is testable and reusable.
* Use `datetime.fromisoformat()` or `dateutil.parser.parse()` to handle datetime parsing.
* Booleans must recognize `"true"`, `"false"`, `"1"`, `"0"` (case-insensitive).
* Trim whitespace from CSV field values before type conversion.
* Must support Unicode characters in values and field names.


## **Function Signature(s) and/or Class Definition(s)**

```python
class CSVTransformer:
    def __init__(self, schema: Dict[str, Dict], mode: str = "flat", group_by: Optional[str] = None,
                 delimiter: str = ",", on_error: str = "raise"):
        ...

    def transform(self, csv_path: str) -> Union[List[Dict], Dict[str, List[Dict]]]:
        """Transforms the CSV at the given path into JSON based on the configuration."""
        ...
```


## **Edge Cases**

* Empty CSV: Return empty list or empty dict depending on mode
* Missing required columns: Raise or log based on `on_error`
* Extra columns not in schema: Ignore them silently
* Duplicate group\_by values in `grouped` mode: Append to the group list
* Nested field names like `meta.info.level` should produce multi-level nesting


## **Constraints**

* Must be implemented using only Python 3.7+ standard library (no Pandas)
* Should not read the entire file into memory for large CSVs; process line-by-line
* Schema must be valid before transformation begins
* Maximum CSV size supported: 100 MB
* Maximum number of columns: 100
* Nested fields should not exceed depth of 5 levels

In [ ]:
# code

"""CSVTransformer: A Python class for transforming CSV files into JSON objects
with schema mapping, type enforcement, and mode-based output formatting."""

import csv
import logging
import os
from datetime import datetime
from typing import Dict, List, Optional, Union

try:
    from dateutil.parser import parse as iso_parse
except ImportError:
    iso_parse = None


class CSVTransformer:
    """
    A class to transform CSV files into JSON objects with schema mapping,
    type enforcement, and mode-based output formatting.

    Supports 'flat', 'grouped', and 'nested' modes with error handling
    strategies and constraints on input size and structure.
    """

    MAX_COLUMNS = 100
    MAX_FILE_SIZE_BYTES = 100 * 1024 * 1024  # 100 MB
    MAX_NESTING_DEPTH = 5

    def __init__(
        self,
        schema: Dict[str, Dict],
        mode: str = "flat",
        group_by: Optional[str] = None,
        delimiter: str = ",",
        on_error: str = "raise",
    ):
        """
        Initialize the transformer.

        Args:
            schema (Dict[str, Dict]): Mapping of CSV fields to output fields
                and expected types.
            mode (str): One of "flat", "grouped", "nested".
            group_by (Optional[str]): Field name to group by (for 'grouped' mode).
            delimiter (str): CSV delimiter character.
            on_error (str): Error handling mode: "raise", "skip", or "log".

        Raises:
            ValueError: If `mode` or `on_error` is invalid, or if required
                parameters are missing.
        """
        valid_modes = {"flat", "grouped", "nested"}
        if mode not in valid_modes:
            raise ValueError(f"Invalid mode '{mode}'. Must be one of {valid_modes}.")

        if on_error not in {"raise", "skip", "log"}:
            raise ValueError("on_error must be one of 'raise', 'skip', or 'log'.")

        if mode == "grouped" and not group_by:
            raise ValueError("`group_by` must be provided when mode is 'grouped'.")

        self.schema = schema
        self.mode = mode
        self.group_by = group_by
        self.delimiter = delimiter
        self.on_error = on_error

        self._validate_schema()

    def _validate_schema(self):
        """
        Validate the schema before processing.

        Ensures target_field and type presence, nesting depth limits,
        and no duplicate target_field paths.

        Raises:
            ValueError: If schema is invalid.
        """
        target_paths = set()
        for field, mapping in self.schema.items():
            if "target_field" not in mapping or "type" not in mapping:
                raise ValueError(
                    f"Schema mapping for '{field}' must include "
                    "'target_field' and 'type'."
                )
            path = mapping["target_field"].split(".")
            if len(path) > self.MAX_NESTING_DEPTH:
                raise ValueError(
                    f"Nested target_field '{mapping['target_field']}' exceeds "
                    f"max depth {self.MAX_NESTING_DEPTH}."
                )
            if mapping["target_field"] in target_paths:
                raise ValueError(
                    f"Duplicate target_field '{mapping['target_field']}' in schema."
                )
            target_paths.add(mapping["target_field"])

    def _check_file_constraints(self, file_path: str):
        """
        Check file size and column count constraints.

        Args:
            file_path (str): Path to the CSV file.

        Raises:
            ValueError: If file size or column count exceeds limits.
        """
        file_size = os.path.getsize(file_path)
        if file_size > self.MAX_FILE_SIZE_BYTES:
            raise ValueError(
                f"CSV file size {file_size} exceeds maximum allowed "
                f"{self.MAX_FILE_SIZE_BYTES} bytes."
            )

        with open(file_path, newline="", encoding="utf-8") as csvfile:
            reader = csv.reader(csvfile, delimiter=self.delimiter)
            try:
                header = next(reader)
            except StopIteration:
                # Empty file: no columns
                return
            if len(header) > self.MAX_COLUMNS:
                raise ValueError(
                    f"CSV has {len(header)} columns which exceeds the max "
                    f"allowed {self.MAX_COLUMNS}."
                )

    def transform(self, csv_path: str) -> Union[List[Dict], Dict[str, List[Dict]]]:
        """
        Transform the CSV file content to JSON object(s) per configuration.

        Returns flattened dicts for 'flat' mode,
        nested dicts for 'nested' mode,
        and grouped dict for 'grouped' mode.

        Args:
            csv_path (str): Path to the CSV file.

        Returns:
            Union[List[Dict], Dict[str, List[Dict]]]: Transformed JSON data.

        Raises:
            FileNotFoundError: If the CSV file does not exist.
            ValueError: If file constraints or schema validation fail.
        """
        self._check_file_constraints(csv_path)

        json_output = []

        try:
            with open(csv_path, newline="", encoding="utf-8") as csvfile:
                csv_reader = csv.DictReader(csvfile, delimiter=self.delimiter)
                if csv_reader.fieldnames is None:
                    return [] if self.mode != "grouped" else {}

                missing_cols = [
                    col for col in self.schema if col not in csv_reader.fieldnames
                ]
                if missing_cols:
                    msg = (
                        f"CSV file is missing required columns: {missing_cols}."
                    )
                    if self.on_error == "raise":
                        raise ValueError(msg)
                    elif self.on_error == "log":
                        logging.error(msg)

                for row in csv_reader:
                    try:
                        transformed_row = self._process_row(row)
                        if transformed_row is not None:
                            if self.mode == "flat":
                                transformed_row = self._flatten_dict(transformed_row)
                            json_output.append(transformed_row)
                    except Exception as e:
                        if self.on_error == "raise":
                            raise
                        elif self.on_error == "log":
                            logging.error(f"Error processing row {row}: {e}")
                        if self.on_error in {"skip", "log"}:
                            continue

            if self.mode == "grouped":
                return self._group_rows(json_output)
            else:
                return json_output

        except FileNotFoundError as e:
            raise FileNotFoundError(f"CSV file not found at path '{csv_path}'.") from e

    def _process_row(self, row: Dict[str, str]) -> Optional[Dict]:
        """
        Process a single CSV row and apply schema mapping and type casting.

        Args:
            row (Dict[str, str]): A CSV row dictionary.

        Returns:
            Optional[Dict]: Transformed row or None if invalid/skipped.

        Raises:
            KeyError: If required field is missing and on_error='raise'.
            ValueError, TypeError: If type casting fails and on_error='raise'.
        """
        transformed_row = {}

        for field_name, mapping in self.schema.items():
            raw_value = row.get(field_name)

            if raw_value is None or raw_value.strip() == "":
                if self.on_error == "raise":
                    raise KeyError(f"Missing or empty column '{field_name}' in row.")
                else:
                    continue

            raw_value = raw_value.strip()
            try:
                casted_value = self._cast_type(raw_value, mapping["type"])
                self._construct_mapped_field(
                    transformed_row, mapping["target_field"], casted_value
                )
            except (ValueError, TypeError) as e:
                if self.on_error == "raise":
                    raise
                elif self.on_error == "log":
                    logging.error(f"Type casting error for row {row}: {e}")
                continue

        if not transformed_row:
            return None
        return transformed_row

    def _construct_mapped_field(self, base: Dict, field_path: str, value):
        """
        Construct nested dictionaries for dot-separated field paths.

        Args:
            base (Dict): Base dictionary to insert into.
            field_path (str): Dot-separated target field path.
            value: Value to set at the nested path.

        Raises:
            ValueError: If nesting depth exceeds limits.
        """
        parts = field_path.split(".")
        if len(parts) > self.MAX_NESTING_DEPTH:
            raise ValueError(
                f"Field path '{field_path}' exceeds max nesting depth "
                f"of {self.MAX_NESTING_DEPTH}."
            )
        current = base
        for part in parts[:-1]:
            if part not in current or not isinstance(current[part], dict):
                current[part] = {}
            current = current[part]
        current[parts[-1]] = value

    def _cast_type(self, value: str, target_type: type):
        """
        Cast a string value to a target type with robust parsing.

        Args:
            value (str): The input string value.
            target_type (type): Target Python type (int, float, bool, datetime, str).

        Returns:
            The value cast to the target type.

        Raises:
            ValueError, TypeError: If conversion fails.
        """
        if target_type == int:
            return int(value)
        elif target_type == float:
            return float(value)
        elif target_type == bool:
            lower_val = value.lower()
            if lower_val in {"true", "1"}:
                return True
            elif lower_val in {"false", "0"}:
                return False
            else:
                raise ValueError(f"Invalid boolean value '{value}'.")
        elif target_type == datetime:
            if iso_parse:
                try:
                    return iso_parse(value)
                except Exception as e:
                    raise ValueError(
                        f"Invalid ISO8601 datetime value '{value}': {e}"
                    ) from e
            else:
                try:
                    return datetime.fromisoformat(value)
                except Exception as e:
                    raise ValueError(
                        f"Invalid ISO8601 datetime value '{value}': {e}"
                    ) from e
        elif target_type == str:
            return value
        else:
            raise TypeError(f"Unsupported target type '{target_type}'.")

    def _group_rows(self, rows: List[Dict]) -> Dict[str, List[Dict]]:
        """
        Group rows by the group_by field.

        Args:
            rows (List[Dict]): List of transformed rows.

        Returns:
            Dict[str, List[Dict]]: Grouped dictionary.

        Raises:
            ValueError: If group_by field is missing in any row and on_error='raise'.
        """
        grouped = {}
        for row in rows:
            key = self._extract_group_key(row)
            if key is None:
                msg = f"Missing group_by key '{self.group_by}' in row {row}."
                if self.on_error == "raise":
                    raise ValueError(msg)
                elif self.on_error == "log":
                    logging.error(msg)
                continue
            grouped.setdefault(key, []).append(row)
        return grouped

    def _extract_group_key(self, row: Dict) -> Optional[str]:
        """
        Extract the group_by key from a possibly nested row.

        Args:
            row (Dict): Transformed row.

        Returns:
            Optional[str]: Group key or None if not found.
        """
        parts = self.group_by.split(".")
        current = row
        for part in parts:
            if not isinstance(current, dict) or part not in current:
                return None
            current = current[part]
        return str(current) if current is not None else None

    def _flatten_dict(self, d: Dict, parent_key: str = "", sep: str = ".") -> Dict:
        """
        Flatten a nested dictionary.

        Args:
            d (Dict): Dictionary to flatten.
            parent_key (str): Prefix key for recursion.
            sep (str): Separator between keys.

        Returns:
            Dict: Flattened dictionary with dot-separated keys.
        """
        items = []
        for k, v in d.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            if isinstance(v, dict):
                items.extend(self._flatten_dict(v, new_key, sep=sep).items())
            else:
                items.append((new_key, v))
        return dict(items)


In [ ]:
# tests
"""Unittests for CSVTransformer: validate modes, casting, schema and errors."""

import unittest
import tempfile
import os
from datetime import datetime
from main import CSVTransformer


class TestCSVTransformer(unittest.TestCase):
    """Unittests for the CSVTransformer class."""

    def setUp(self):
        """Set up temporary CSV files and schema for tests."""
        self.schema = {
            "id": {"target_field": "user.id", "type": int},
            "name": {"target_field": "user.name", "type": str},
            "active": {"target_field": "status.active", "type": bool},
            "created": {"target_field": "meta.created", "type": datetime},
        }

        self.csv_data = (
            "id,name,active,created\n"
            "1,Alice,true,2023-01-01T10:00:00Z\n"
            "2,Bob,false,2023-01-02T12:30:00Z\n"
        )

        self.temp_file = tempfile.NamedTemporaryFile(mode="w+", delete=False)
        self.temp_file.write(self.csv_data)
        self.temp_file.close()

    def tearDown(self):
        """Clean up the temporary file."""
        os.unlink(self.temp_file.name)

    def test_flat_mode(self):
        """Test transformation in flat mode."""
        transformer = CSVTransformer(self.schema, mode="flat")
        result = transformer.transform(self.temp_file.name)
        self.assertEqual(len(result), 2)
        self.assertIn("user.id", result[0])

    def test_nested_mode(self):
        """Test transformation in nested mode."""
        transformer = CSVTransformer(self.schema, mode="nested")
        result = transformer.transform(self.temp_file.name)
        self.assertEqual(result[0]["user"]["id"], 1)

    def test_grouped_mode(self):
        """Test transformation in grouped mode by name."""
        schema = {
            "name": {"target_field": "name", "type": str},
            "id": {"target_field": "id", "type": int},
        }
        csv_data = "id,name\n1,Alice\n2,Alice\n3,Bob\n"
        file = tempfile.NamedTemporaryFile(mode="w+", delete=False)
        file.write(csv_data)
        file.close()

        transformer = CSVTransformer(schema, mode="grouped", group_by="name")
        result = transformer.transform(file.name)
        os.unlink(file.name)
        self.assertEqual(len(result["Alice"]), 2)

    def test_invalid_mode(self):
        """Test initialization with an invalid mode."""
        with self.assertRaises(ValueError):
            CSVTransformer(self.schema, mode="invalid")

    def test_invalid_on_error(self):
        """Test initialization with invalid on_error strategy."""
        with self.assertRaises(ValueError):
            CSVTransformer(self.schema, on_error="fail")

    def test_grouped_without_group_by(self):
        """Test grouped mode without group_by."""
        with self.assertRaises(ValueError):
            CSVTransformer(self.schema, mode="grouped")

    def test_exceed_max_columns(self):
        """Test column limit enforcement."""
        csv_data = ",".join([f"col{i}" for i in range(101)]) + "\n"
        file = tempfile.NamedTemporaryFile(mode="w+", delete=False)
        file.write(csv_data)
        file.close()
        transformer = CSVTransformer({}, mode="flat")

        with self.assertRaises(ValueError):
            transformer.transform(file.name)
        os.unlink(file.name)

    def test_empty_file(self):
        """Test handling of empty file."""
        file = tempfile.NamedTemporaryFile(mode="w+", delete=False)
        file.close()
        transformer = CSVTransformer(self.schema, mode="flat")
        result = transformer.transform(file.name)
        os.unlink(file.name)
        self.assertEqual(result, [])

    def test_missing_required_column(self):
        """Test missing column error behavior."""
        csv_data = "id,created\n1,2022-01-01T00:00:00Z\n"
        file = tempfile.NamedTemporaryFile(mode="w+", delete=False)
        file.write(csv_data)
        file.close()
        transformer = CSVTransformer(
            self.schema, mode="flat", on_error="skip"
        )
        result = transformer.transform(file.name)
        os.unlink(file.name)
        self.assertEqual(len(result), 1)

    def test_casting_error_skip(self):
        """Test skipping rows with type casting errors."""
        csv_data = "id,name,active,created\nX,Alice,true,invalid\n"
        file = tempfile.NamedTemporaryFile(mode="w+", delete=False)
        file.write(csv_data)
        file.close()
        transformer = CSVTransformer(
            self.schema, mode="flat", on_error="skip"
        )
        result = transformer.transform(file.name)
        os.unlink(file.name)
        self.assertEqual(
            result,
            [{'user.name': 'Alice', 'status.active': True}]
        )

    def test_casting_error_raise(self):
        """Test raising exception on casting errors."""
        csv_data = "id,name,active,created\nX,Alice,true,invalid\n"
        file = tempfile.NamedTemporaryFile(mode="w+", delete=False)
        file.write(csv_data)
        file.close()
        transformer = CSVTransformer(
            self.schema, mode="flat", on_error="raise"
        )

        with self.assertRaises(Exception):
            transformer.transform(file.name)
        os.unlink(file.name)

    def test_boolean_true_casting(self):
        """Test valid boolean string conversion."""
        transformer = CSVTransformer(
            {"active": {"target_field": "a", "type": bool}}
        )
        row = {"active": "true"}
        result = transformer._process_row(row)
        self.assertTrue(result["a"])

    def test_boolean_invalid_casting(self):
        """Test invalid boolean string conversion raises."""
        transformer = CSVTransformer(
            {"active": {"target_field": "a", "type": bool}},
            on_error="raise"
        )
        row = {"active": "yes"}
        with self.assertRaises(ValueError):
            transformer._process_row(row)

    def test_datetime_casting(self):
        """Test valid datetime parsing."""
        transformer = CSVTransformer(
            {"created": {"target_field": "meta.created", "type": datetime}}
        )
        row = {"created": "2024-01-01T10:00:00Z"}
        result = transformer._process_row(row)
        self.assertIsInstance(result["meta"]["created"], datetime)

    def test_unsupported_type_cast(self):
        """Test unsupported type raises TypeError."""
        transformer = CSVTransformer(
            {"x": {"target_field": "x", "type": list}}
        )
        row = {"x": "[1,2]"}
        with self.assertRaises(TypeError):
            transformer._process_row(row)

    def test_flatten_nested_dict(self):
        """Test flattening a nested dictionary."""
        transformer = CSVTransformer(self.schema)
        nested = {"a": {"b": {"c": 1}}}
        flat = transformer._flatten_dict(nested)
        self.assertEqual(flat["a.b.c"], 1)

    def test_missing_field_skipped(self):
        """Test skipping missing field when on_error is skip."""
        transformer = CSVTransformer(self.schema, on_error="skip")
        row = {"id": "1", "name": ""}
        result = transformer._process_row(row)
        self.assertEqual(result.get("user", {}).get("id"), 1)

    def test_group_key_extraction(self):
        """Test group key extraction from nested path."""
        transformer = CSVTransformer(
            self.schema, mode="grouped", group_by="user.name"
        )
        row = {"user": {"name": "Alice"}}
        key = transformer._extract_group_key(row)
        self.assertEqual(key, "Alice")

    def test_construct_mapped_field(self):
        """Test building nested path."""
        transformer = CSVTransformer(self.schema)
        base = {}
        transformer._construct_mapped_field(base, "a.b.c", 123)
        self.assertEqual(base["a"]["b"]["c"], 123)


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/e48a6f45-1a99-4b32-ab39-0f4fbc163e5f?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F74834%2Fview

#### Model code:

```python
import csv
import json
from typing import List, Dict, Union, Optional
from datetime import datetime
import logging

class CSVTransformer:
    def __init__(self, schema: Dict[str, Dict], mode: str = "flat", group_by: Optional[str] = None,
                 delimiter: str = ",", on_error: str = "raise"):
        self.schema = schema
        self.mode = mode
        self.group_by = group_by
        self.delimiter = delimiter
        if on_error not in ["raise", "skip", "log"]:
            raise ValueError("on_error must be one of 'raise', 'skip', or 'log'")
        self.on_error = on_error

    def transform(self, csv_path: str) -> Union[List[Dict], Dict[str, List[Dict]]]:
        json_output = []

        try: # <Issue>: There is no upfront validation of the schema to check if required columns exist or if the schema itself is well-formed, risking runtime errors.
            with open(csv_path, newline='', encoding='utf-8') as csvfile:
                csv_reader = csv.DictReader(csvfile, delimiter=self.delimiter)
                for row in csv_reader:
                    try:
                        transformed_row = self.process_row(row)
                        if transformed_row:
                            json_output.append(transformed_row)
                    except Exception as e:
                        self.handle_error(e, row)

                if self.mode == "grouped" and self.group_by: # <Issue>: The code does not handle mode == "nested" differently; it simply returns a flat or grouped list. The problem specifies nested mode should create nested objects based on prefixes distinctly.
                    return self.group_rows(json_output)
                return json_output
        except FileNotFoundError:
            raise FileNotFoundError(f"CSV file at path '{csv_path}' not found.")

    def process_row(self, row: Dict[str, str]) -> Optional[Dict]:
        transformed_row = {}
        
        for field_name, mapping in self.schema.items():
            original_value = row.get(field_name, None)
            
            if original_value is None and self.on_error == "raise":
                raise KeyError(f"Missing required column '{field_name}' in row.")
            if original_value is None:
                continue

            try:
                transformed_value = self.cast_type(original_value.strip(), mapping['type'])
                self.construct_mapped_field(transformed_row, mapping['target_field'], transformed_value)
            except (ValueError, TypeError) as e:
                self.handle_error(e, row)
        
        return transformed_row

    def construct_mapped_field(self, base: Dict, field_path: str, value): # <Issue>: There is no check or limitation on the nesting depth, allowing arbitrarily deep nested structures which violate the constraint.
        parts = field_path.split('.')
        for part in parts[:-1]:
            base = base.setdefault(part, {})
        base[parts[-1]] = value

    def cast_type(self, value: str, target_type: type):
        if target_type == int:
            return int(value)
        elif target_type == float:
            return float(value)
        elif target_type == bool:
            return value.lower() in ["true", "1"]
        elif target_type == datetime:
            return datetime.fromisoformat(value)
        elif target_type == str:
            return value
        else:
            raise TypeError(f"Unsupported type specified: {target_type}")

    def group_rows(self, rows: List[Dict]) -> Dict[str, List[Dict]]:
        grouped_data = {}
        for row in rows:
            group_key = row.get(self.group_by)
            if group_key is not None:
                if group_key not in grouped_data:
                    grouped_data[group_key] = []
                grouped_data[group_key].append(row)
        return grouped_data

    def handle_error(self, error: Exception, row: Dict):
        if self.on_error == "raise":
            raise error
        elif self.on_error == "log":
            logging.error(f"Error processing row {row}: {error}")

```